In [15]:
#importing libraries

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split,DataLoader
import torch.optim as optim

In [5]:
# setting file path
train_filepath = "../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
test_filepath = "../input/asl-alphabet/asl_alphabet_test"

In [6]:
# defining transforms
train_transforms = transforms.Compose([
    transforms.ToTensor()
])

In [7]:
#Data Loading 
train_dataset = datasets.ImageFolder(train_filepath, transform=train_transforms)
print("Train Dataset Info:\n")
train_dataset

Train Dataset Info:



Dataset ImageFolder
    Number of datapoints: 87000
    Root location: ../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [9]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=100, 
                                               shuffle=True)

In [10]:
train_label = train_dataloader.dataset.classes
print("Labels :\t",train_label,"\nLength of the label list :",len(train_label))

Labels :	 ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space'] 
Length of the label list : 29


In [12]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# # data, labels = data.to(device), labels.to(device)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [16]:
model_resnet = torchvision.models.resnet50(pretrained=True)
model_resnet

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
for param in model_resnet.parameters():
    param.requires_grad = False

In [20]:
in_features = model_resnet.fc.in_features
model_resnet.fc = torch.nn.Linear(in_features, 29)
model_resnet


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [21]:
!pip install torchsummary

In [22]:
from torchsummary import summary
model_resnet.to(device)
summary(model_resnet, (3, 200, 200), batch_size=100)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [100, 64, 100, 100]           9,408
       BatchNorm2d-2        [100, 64, 100, 100]             128
              ReLU-3        [100, 64, 100, 100]               0
         MaxPool2d-4          [100, 64, 50, 50]               0
            Conv2d-5          [100, 64, 50, 50]           4,096
       BatchNorm2d-6          [100, 64, 50, 50]             128
              ReLU-7          [100, 64, 50, 50]               0
            Conv2d-8          [100, 64, 50, 50]          36,864
       BatchNorm2d-9          [100, 64, 50, 50]             128
             ReLU-10          [100, 64, 50, 50]               0
           Conv2d-11         [100, 256, 50, 50]          16,384
      BatchNorm2d-12         [100, 256, 50, 50]             512
           Conv2d-13         [100, 256, 50, 50]          16,384
      BatchNorm2d-14         [100, 256,

In [25]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_resnet.parameters(), lr=0.01)

In [28]:
num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    for batch_idx , (data,labels) in enumerate(train_dataloader):
        data = data.to(device=device)
        labels = labels.to(device= device)
        data, labels = data.cuda(), labels.cuda()
        
        #forward
        scores = model_resnet(data)
        loss = criterion(scores, labels)

        #backprop
        optimizer.zero_grad()
        loss.backward()

        #gradient descent
        optimizer.step()
        
        #performance
        prediction = torch.max(scores,1)[1]
    
        correct += (prediction == labels).cpu().sum().numpy()

    print("Epoch :",epoch,"\tLoss :",loss.data,"\tAccuracy :",(correct/len(train_dataloader.dataset))*100)        

Epoch : 0 	Loss : tensor(0.1924, device='cuda:0') 	Accuracy : 85.64137931034482
Epoch : 1 	Loss : tensor(0.3782, device='cuda:0') 	Accuracy : 91.22298850574713
Epoch : 2 	Loss : tensor(0.4589, device='cuda:0') 	Accuracy : 92.50344827586207
Epoch : 3 	Loss : tensor(0.1326, device='cuda:0') 	Accuracy : 93.1264367816092
Epoch : 4 	Loss : tensor(0.1803, device='cuda:0') 	Accuracy : 93.71494252873563
